In [1]:
import os

In [2]:
%pwd

'c:\\Users\\shiva\\Desktop\\ml project1\\Cement_strength_ml_project\\research'

In [3]:
os.chdir("../")

In [4]:
%pwd

'c:\\Users\\shiva\\Desktop\\ml project1\\Cement_strength_ml_project'

In [ ]:
from dataclasses import dataclass
from pathlib import Path


@dataclass(frozen=True)
class ModelTrainerConfig:
    root_dir: Path
    train_data_path: Path
    test_data_path: Path
    model_name: str
    max_depth: int
    max_features: str
    min_samples_leaf: int
    min_samples_split: int
    n_estimators: int
    target_column: str



In [ ]:
from cementmlProject.constants import *
from cementmlProject.utils.common import read_yaml, create_directories

In [ ]:
class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH,
        schema_filepath = SCHEMA_FILE_PATH):

        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        self.schema = read_yaml(schema_filepath)

        create_directories([self.config.artifacts_root])


    
    def get_model_trainer_config(self) -> ModelTrainerConfig:
        config = self.config.model_trainer
        params = self.params.RandomForest
        schema =  self.schema.TARGET_COLUMN

        create_directories([config.root_dir])

        model_trainer_config = ModelTrainerConfig(
            root_dir=config.root_dir,
            train_data_path = config.train_data_path,
            test_data_path = config.test_data_path,
            model_name = config.model_name,
            max_depth=params.max_depth,
            max_features=params.max_features,
            min_samples_leaf=params.min_samples_leaf,
            min_samples_split=params.min_samples_split,
            n_estimators=params.n_estimators,
            target_column = schema.name
            
        )

        return model_trainer_config

In [ ]:
import pandas as pd
import os
from cementmlProject import logger
from sklearn.ensemble import RandomForestRegressor
import joblib

In [ ]:
class ModelTrainer:
    def __init__(self, config: ModelTrainerConfig):
        self.config = config


    
    def train(self):
        train_data = pd.read_csv(self.config.train_data_path)
        test_data = pd.read_csv(self.config.test_data_path)


        train_x = train_data.drop([self.config.target_column], axis=1)
        test_x = test_data.drop([self.config.target_column], axis=1)
        train_y = train_data[[self.config.target_column]]
        test_y = test_data[[self.config.target_column]]


        RF = RandomForestRegressor(max_depth=self.config.max_depth, max_feature=self.config.max_features,min_samples_leaf=self.config.min_samples_leaf,min_samples_split=self.config.min_samples_split,n_estimators=self.config.n_estimators, random_state=42)
        RF.fit(train_x, train_y)

        joblib.dump(RF, os.path.join(self.config.root_dir, self.config.model_name))

In [ ]:
try:
    config = ConfigurationManager()
    model_trainer_config = config.get_model_trainer_config()
    model_trainer_config = ModelTrainer(config=model_trainer_config)
    model_trainer_config.train()
except Exception as e:
    raise e